### DFSA License Recommendation Agent

# Prompts Templating for the License Recommendation Agent - LangChain Integration

## Overview

The License Recommendation Agent is a sophisticated, AI-powered assistant designed to help new applicants and regulated entities navigate the DFSA licensing process. Built using Python for backend processing, JavaScript for frontend interactions, and REST APIs for communication, the agent provides real-time license recommendations, eligibility checks, and regulatory pathway guidance.

## Purpose

The License Recommendation Agent is designed to:

* **Assist New Applicants**: Help businesses identify the correct DFSA license based on their sector, services, and activities.
* **Help Regulated Entities**: Guide regulated businesses in modifying or renewing their licenses according to their updated operations.
* **Ensure Compliance**: Validate the eligibility of applicants for the recommended license type and guide them through the regulatory process.
* **Increase Efficiency**: Automate the process of selecting the correct license and validating eligibility, reducing manual errors and improving overall compliance.

## Key Features

| Feature | Description |
|---------|-------------|
| License Mapping | Recommends the most suitable DFSA license based on user input (business sector, activities, etc.). |
| Eligibility Check | Verifies whether the user meets the eligibility criteria for the suggested license. |
| Regulatory Pathway Guidance | Provides step-by-step guidance through the entire licensing process, including documentation and approvals. |
| Real-Time Recommendations | Dynamic recommendations using data sourced directly from the DFSA Licensing Database. |

## Prompting Strategy

Prompts are fundamental to the License Recommendation Agent's functionality. Through natural language instructions, we guide the LLM to provide accurate license recommendations, check eligibility, and deliver regulatory guidance. LangChain enables us to build dynamic prompting pipelines that adapt based on user input, business sector, and eligibility status.

In this notebook, we'll explore how to structure prompts for the License Recommendation Agent using **R**etrieval **A**ugmented **G**eneration (RAG) patterns combined with the DFSA licensing database.

---

> ⚠️ We will be using OpenAI for this example allowing us to run everything via API. If you would like to use Ollama instead, please see the [Ollama version](https://github.com/aurelio-labs/langchain-course/blob/main/notebooks/ollama/02-prompts-ollama.ipynb) of this example.

---

## Basic Prompting for License Recommendations

For the License Recommendation Agent, we structure prompts with the following core components:

* **System Instructions**: Defines the agent's role as a DFSA licensing expert, how it should approach recommendations, and its behavior guidelines.

* **Context**: Retrieved from the DFSA Licensing Database, containing license types, eligibility criteria, and regulatory requirements relevant to the user's query.

* **User Query**: The applicant's question about licensing, eligibility, or regulatory pathways.

* **Agent Response**: The recommended license, eligibility status, and next steps in the licensing process.

Below is an example of how a License Recommendation prompt may look:

```
You are a DFSA Licensing Expert Assistant.                       }
Provide accurate license recommendations based on the            }--->  (System Instructions)
applicant's business sector and activities.                      }

DFSA License Database:                                           }
- Financial Services License: For companies providing             }
  investment services, wealth management, etc.                   }
- Insurance License: For insurance brokers and agents.           }--->  (Context from DFSA DB)
- Freezone License: For businesses operating in DIFC.            }
Eligibility: Minimum capital of AED 500,000.                     }

User Query: I run a financial services company. What license      }--->  (User Query)
do I need?

Recommendation:                                                  }--->  (Agent Response)
```

Here we can see how the License Recommendation Agent approaches a user query. The system instructions define the agent's expertise, the context provides relevant DFSA licensing information, the user query is the applicant's question, and the agent response includes the recommended license and next steps. This structure ensures accurate, compliant recommendations based on real DFSA data.

In [1]:
# License Recommendation Agent - System Prompt Template
system_prompt = """
You are an expert DFSA Licensing Assistant. Your role is to:
1. Recommend the most suitable DFSA license based on the applicant's business sector and activities.
2. Verify eligibility criteria for the recommended license.
3. Provide step-by-step regulatory pathway guidance.
4. Ensure all recommendations comply with DFSA regulations.

If you cannot determine the appropriate license based on the provided information,
ask clarifying questions about the applicant's business activities and sector.

Always reference the DFSA Licensing Database for accurate information.
"""

# RAG Context Template for License Recommendations
rag_prompt = """
Based on the DFSA Licensing Database context below, provide a license recommendation.
If you cannot determine the appropriate license, ask clarifying questions.

Context: {context}
"""

LangChain uses a `ChatPromptTemplate` object to format the various prompt types into a single list which will be passed to our LLM. For the License Recommendation Agent, we structure it with system instructions and user queries:

In [2]:
from langchain.prompts import ChatPromptTemplate

# License Recommendation Agent - ChatPromptTemplate
license_prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{query}"),
])

# RAG-enabled License Recommendation Template
rag_license_template = ChatPromptTemplate.from_messages([
    ("system", rag_prompt),
    ("user", "{query}"),
])

When we call the template it will expect us to provide the `query` variable. For RAG-enabled templates, we also provide `context` from the DFSA Licensing Database. LangChain interprets curly-bracket syntax (ie `{context}` and `{query}`) as indicating a dynamic variable that we expect to be inserted at query time. We can see that these variables have been picked up by our template object by viewing its `input_variables` attribute:

In [3]:
license_prompt_template.input_variables

['query']

We can also view the structure of the messages (currently _prompt templates_) that the `ChatPromptTemplate` will construct by viewing the `messages` attribute. For the License Recommendation Agent:

In [4]:
license_prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an expert DFSA Licensing Assistant...'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'), additional_kwargs={})]

From this, we can see that each tuple provided when using `ChatPromptTemplate.from_messages` becomes an individual prompt template itself. Within each of these tuples, the first value defines the _role_ of the message, which is typically `system`, `human`, or `ai`. For the License Recommendation Agent, we use explicit template definitions:

In [5]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Explicit License Recommendation Template
license_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("{query}"),
])

# Explicit RAG License Template
rag_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(rag_prompt),
    HumanMessagePromptTemplate.from_template("{query}"),
])

We can see the structure of this new chat prompt template is identical to our previous:

In [6]:
prompt_template

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.                 \nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])

### Invoking the License Recommendation Agent with Templates

We've defined our prompt templates for the License Recommendation Agent. Now let's initialize the Azure OpenAI LLM and create a pipeline to process license recommendation queries.

The License Recommendation Agent uses Azure OpenAI for backend processing. Ensure your Azure OpenAI credentials are configured in your environment variables.

In [7]:
import os
from getpass import getpass

# Azure OpenAI Configuration
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT") or getpass("Enter Azure OpenAI Endpoint: ")
azure_key = os.getenv("AZURE_OPENAI_API_KEY") or getpass("Enter Azure OpenAI API Key: ")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4-32k")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")

In [8]:
from langchain_openai import AzureChatOpenAI

# Initialize Azure OpenAI LLM for License Recommendations
llm = AzureChatOpenAI(
    temperature=0.0,  # Deterministic for accurate license recommendations
    azure_deployment=deployment_name,
    azure_endpoint=azure_endpoint,
    api_key=azure_key,
    api_version=api_version
)

For the License Recommendation Agent, we use `temperature=0.0` to ensure deterministic, accurate recommendations. This prevents hallucinations and ensures compliance with DFSA regulations.

We create a pipeline by connecting our `license_prompt_template` and `llm` using **L**ang**C**hain **E**xpression **L**anguage (LCEL), which uses the `|` operator to chain components together.

In [9]:
# Create License Recommendation Pipeline
license_pipeline = license_prompt_template | llm

# Create RAG-enabled License Recommendation Pipeline
rag_license_pipeline = rag_license_template | llm

Now let's define a user query and relevant DFSA licensing context, then invoke our pipeline to get a license recommendation.

In [10]:
# DFSA License Database Context
dfsa_context = """DFSA License Types:
1. Financial Services License: For investment services, wealth management, and financial advisory.
   - Minimum Capital: AED 500,000
   - Eligibility: Regulated entity with compliance framework

2. Insurance License: For insurance brokers, agents, and underwriters.
   - Minimum Capital: AED 250,000
   - Eligibility: Insurance industry experience required

3. Freezone License: For businesses operating in DIFC (Dubai International Financial Centre).
   - Minimum Capital: AED 100,000
   - Eligibility: Any business sector eligible

4. Crowdfunding License: For peer-to-peer lending and crowdfunding platforms.
   - Minimum Capital: AED 1,000,000
   - Eligibility: Technology platform with compliance systems
"""

# Example user query
user_query = "I run a financial services company with AED 600,000 in capital. What license do I need?"

In [11]:
# Invoke the License Recommendation Pipeline
license_pipeline.invoke({"query": user_query})

AIMessage(content='Based on your financial services company with AED 600,000 in capital, I recommend the Financial Services License. Your capital exceeds the minimum requirement of AED 500,000, making you eligible. Next steps: 1) Prepare compliance framework documentation, 2) Submit application to DFSA, 3) Undergo regulatory review, 4) Receive license approval.')

Our LLM pipeline is able to consume the information from the `context` and use it to answer the user's `query`. Ofcourse, we would not usually be feeding in both a question and an answer into an LLM manually. Typically, the `context` would be retrieved from a vector database, via web search, or from elsewhere. We will cover this use-case in full and build a functional RAG pipeline in a future chapter.

## Few Shot Prompting for License Recommendations

Few-shot prompting is valuable for the License Recommendation Agent to provide consistent, accurate recommendations. By providing examples of user queries and expected license recommendations, we can improve the agent's ability to handle similar cases.

Let's set up few-shot prompting for license recommendations:

In [12]:
# Few-shot example template for license recommendations
license_example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{query}"),
    ("ai", "{recommendation}"),
])

Now we define examples of license recommendation queries and their expected recommendations:

In [13]:
license_examples = [
    {
        "query": "I run a financial services company with AED 600,000 in capital. What license do I need?",
        "recommendation": "Financial Services License. Your capital exceeds the minimum requirement of AED 500,000. Next steps: 1) Prepare compliance framework, 2) Submit application to DFSA, 3) Undergo regulatory review."
    },
    {
        "query": "I'm an insurance broker looking to operate in Dubai. What license should I apply for?",
        "recommendation": "Insurance License. Minimum capital requirement is AED 250,000. Ensure you have insurance industry experience and compliance systems in place."
    },
    {
        "query": "I want to start a tech startup in the freezone. What are my licensing options?",
        "recommendation": "Freezone License is ideal for your tech startup. Minimum capital is AED 100,000. DIFC offers a business-friendly environment for technology companies."
    },
]

Now we create a FewShotChatMessagePromptTemplate for license recommendations:

In [14]:
from langchain.prompts import FewShotChatMessagePromptTemplate

few_shot_license_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=license_example_prompt,
    examples=license_examples,
)
# Display the formatted few-shot prompt
print(few_shot_prompt.format())

Human: I run a financial services company with AED 600,000 in capital. What license do I need?
AI: Financial Services License. Your capital exceeds the minimum requirement of AED 500,000. Next steps: 1) Prepare compliance framework, 2) Submit application to DFSA, 3) Undergo regulatory review.
Human: I'm an insurance broker looking to operate in Dubai. What license should I apply for?
AI: Insurance License. Minimum capital requirement is AED 250,000. Ensure you have insurance industry experience and compliance systems in place.
Human: I want to start a tech startup in the freezone. What are my licensing options?
AI: Freezone License is ideal for your tech startup. Minimum capital is AED 100,000. DIFC offers a business-friendly environment for technology companies.


Using this we can provide different sets of `examples` or even different individual `example_prompt` templates to the `FewShotChatMessagePromptTemplate` object to build our prompt structure. Let's try an real example where we might use few-shot prompting.

### Few-Shot Example

Using our tiny LLM limits it's ability, so when asking for specific behaviors or structured outputs it can struggle. For example, we'll ask the LLM to summarize the key points about Aurelio AI using markdown and bullet points. Let's see what happens.

In [ ]:
new_system_prompt = """
Answer the user's query based on the context below.                 
If you cannot answer the question using the
provided information answer with "I don't know".

When doing so please
provide the answers to the point helpful engaging tone  .

Context: {context}
"""

prompt_template.messages[0].prompt.template = new_system_prompt

out = pipeline.invoke({"query": query, "context": context}).content
print(out)

# Overview of Aurelio AI

Aurelio AI is an AI company that specializes in developing tools and solutions for AI engineers, particularly in the realm of language AI. 

## Key Focus Areas
- **Language AI**: Expertise in building AI agents and information retrieval systems.
- **Open Source Frameworks**: Development of frameworks like Semantic Router and Semantic Chunkers.
- **AI Platform**: Provides tooling for engineers to facilitate AI development.
- **Development Services**: Assists organizations in bringing their AI technologies to market.

## Achievements
- Became LangChain Experts in September 2024, showcasing their proficiency in the LangChain ecosystem.

In conclusion, Aurelio AI is dedicated to empowering AI engineers through innovative tools, frameworks, and expert services in the field of language AI.


We can display our markdown nicely with `IPython` like so:

In [16]:
from IPython.display import display, Markdown

display(Markdown(out))

# Overview of Aurelio AI

Aurelio AI is an AI company that specializes in developing tools and solutions for AI engineers, particularly in the realm of language AI. 

## Key Focus Areas
- **Language AI**: Expertise in building AI agents and information retrieval systems.
- **Open Source Frameworks**: Development of frameworks like Semantic Router and Semantic Chunkers.
- **AI Platform**: Provides tooling for engineers to facilitate AI development.
- **Development Services**: Assists organizations in bringing their AI technologies to market.

## Achievements
- Became LangChain Experts in September 2024, showcasing their proficiency in the LangChain ecosystem.

In conclusion, Aurelio AI is dedicated to empowering AI engineers through innovative tools, frameworks, and expert services in the field of language AI.

This is not bad, but also not quite the format we wanted. We could try improving our initial prompting instructions, but when this doesn't work we can move on to our few-shot prompting. We want to build something like this:

```
Answer the user's query based on the context below,                 }
if you cannot answer the question using the                         }
provided information answer with "I don't know"                     }
                                                                    }--->  (Rules)
Always answer in markdown format. When doing so please              }
provide headers, short summaries, follow with bullet                }
points, then conclude. Here are some examples:                      }


User: Can you explain gravity?                                      }
AI: ## Gravity                                                      }
                                                                    }
Gravity is one of the fundamental forces in the universe.           }
                                                                    }
### Discovery                                                       }--->  (Example 1)
                                                                    }
* Gravity was first discovered by...                                }
                                                                    }
**To conclude**, Gravity is a fascinating topic and has been...     }
                                                                    }

User: What is the capital of France?                                }
AI: ## France                                                       }
                                                                    }
The capital of France is Paris.                                     }
                                                                    }--->  (Example 2)
### Origins                                                         }
                                                                    }
* The name Paris comes from the...                                  }
                                                                    }
**To conclude**, Paris is highly regarded as one of the...          }

Context: {context}                                                  }--->  (Context)
```

We have already defined our `example_prompt` so now we just change our `examples` to use some examples of a user asking a question and the LLM answering in the exact markdown format we need.

In [17]:
examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]

We feed these into our `FewShotChatMessagePromptTemplate` object:

In [18]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

Our formatted prompt now looks like this:

In [19]:
out = few_shot_prompt.format()

display(Markdown(out))

Human: Can you explain gravity?
AI: ## Gravity

Gravity is one of the fundamental forces in the universe.

### Discovery

* Gravity was first discovered by Sir Isaac Newton in the late 17th century.
* It was said that Newton theorized about gravity after seeing an apple fall from a tree.

### In General Relativity

* Gravity is described as the curvature of spacetime.
* The more massive an object is, the more it curves spacetime.
* This curvature is what causes objects to fall towards each other.

### Gravitons

* Gravitons are hypothetical particles that mediate the force of gravity.
* They have not yet been detected.

**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.


Human: What is the capital of France?
AI: ## France

The capital of France is Paris.

### Origins

* The name Paris comes from the Latin word "Parisini" which referred to a Celtic people living in the area.
* The Romans named the city Lutetia, which means "the place where the river turns".
* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.

**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.



We then pull all of this together with our system prompt and final user query to create our final prompt and feed it into our LLM.

In [20]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", new_system_prompt),
    few_shot_prompt,
    ("user", "{query}"),
])

Now feed this back into our pipeline:

In [21]:
pipeline = prompt_template | llm
out = pipeline.invoke({"query": query, "context": context}).content
display(Markdown(out))

## Aurelio AI Overview

Aurelio AI is an AI company focused on developing tools and services for AI engineers, particularly in the realm of language AI.

### Key Areas of Focus

- **Language AI**: Specializes in creating AI solutions that understand and generate human language.
- **Open Source Frameworks**: Developed notable frameworks like Semantic Router and Semantic Chunkers.
- **AI Platform**: Provides engineers with tools to facilitate AI development.
- **Development Services**: Offers services to help organizations bring their AI technologies to market.

### Expertise

- The team has strong expertise in building AI agents.
- They possess a solid background in information retrieval.

### Recognition

- Became LangChain Experts in September 2024, showcasing their proficiency in the LangChain ecosystem.

**To conclude**, Aurelio AI is dedicated to empowering AI engineers through innovative tools, frameworks, and development services in the language AI space.

We can see that by adding a few examples to our prompt, ie _few-shot prompting_, we can get much more control over the exact structure of our LLM response. As the size of our LLMs increases, the ability of them to follow instructions becomes much greater and they tend to require less explicit prompting as we have shown here. However, even for SotA models like `gpt-4o` few-shot prompting is still a valid technique that can be used if the LLM is struggling to follow our intended instructions.

## Chain of Thought Prompting

We'll take a look at one more commonly used prompting technique called _chain of thought_ (CoT). CoT is a technique that encourages the LLM to think through the problem step by step before providing an answer. The idea being that by breaking down the problem into smaller steps, the LLM is more likely to arrive at the correct answer and we are less likely to see hallucinations.

To implement CoT we don't need any specific LangChain objects, instead we are simply modifying how we instruct our LLM within the system prompt. We will ask the LLM to list the problems that need to be solved, to solve each problem individually, and then to arrive at the final answer.

Let's first test our LLM _without_ CoT prompting.

In [22]:
no_cot_system_prompt = """
Be a helpful assistant and answer the user's question.

You MUST answer the question directly without any other
text or explanation.
"""

no_cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", no_cot_system_prompt),
    ("user", "{query}"),
])

Nowadays most LLMs are trained to use CoT prompting by default, so we actually need to instruct it not to do so for this example which is why we added `"You MUST answer the question directly without any other text or explanation."` to our system prompt.

In [23]:
query = (
    "How many keystrokes are needed to type the numbers from 1 to 500?"
)

no_cot_pipeline = no_cot_prompt_template | llm
no_cot_result = no_cot_pipeline.invoke({"query": query}).content
print(no_cot_result)

The total number of keystrokes needed to type the numbers from 1 to 500 is 1,500.


The actual answer is `1392`, but the LLM _without_ CoT just hallucinates and gives us a guess. Now, we can add explicit CoT prompting to our system prompt to see if we can get a better result.

In [24]:
# Define the chain-of-thought prompt template
cot_system_prompt = """
Be a helpful assistant and answer the user's question.

To answer the question, you must:

- List systematically and in precise detail all
  subproblems that need to be solved to answer the
  question.
- Solve each sub problem INDIVIDUALLY and in sequence.
- Finally, use everything you have worked through to
  provide the final answer.

Context: {context}
"""

cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", cot_system_prompt),
    ("user", "{query}"),
])

cot_pipeline = cot_prompt_template | llm

In [25]:
cot_result = cot_pipeline.invoke({"query": query, "context": context}).content
display(Markdown(cot_result))

To determine how many keystrokes are needed to type the numbers from 1 to 500, we can break this problem down into several subproblems:

1. **Count the numbers from 1 to 9**: These are single-digit numbers.
2. **Count the numbers from 10 to 99**: These are two-digit numbers.
3. **Count the numbers from 100 to 499**: These are three-digit numbers.
4. **Count the number 500**: This is a specific case of a three-digit number.

Now, let's solve each subproblem individually.

### Subproblem 1: Count the numbers from 1 to 9
- There are 9 numbers (1, 2, 3, 4, 5, 6, 7, 8, 9).
- Each number requires 1 keystroke.
- Total keystrokes = 9 numbers * 1 keystroke = **9 keystrokes**.

### Subproblem 2: Count the numbers from 10 to 99
- There are 90 numbers (10 to 99).
- Each number requires 2 keystrokes.
- Total keystrokes = 90 numbers * 2 keystrokes = **180 keystrokes**.

### Subproblem 3: Count the numbers from 100 to 499
- There are 400 numbers (100 to 499).
- Each number requires 3 keystrokes.
- Total keystrokes = 400 numbers * 3 keystrokes = **1200 keystrokes**.

### Subproblem 4: Count the number 500
- The number 500 requires 3 keystrokes.
- Total keystrokes = **3 keystrokes**.

### Final Calculation
Now, we can sum all the keystrokes from each subproblem:
- Keystrokes from 1 to 9: 9
- Keystrokes from 10 to 99: 180
- Keystrokes from 100 to 499: 1200
- Keystrokes for 500: 3

Total keystrokes = 9 + 180 + 1200 + 3 = **1392 keystrokes**.

Thus, the total number of keystrokes needed to type the numbers from 1 to 500 is **1392**.

Now we get a much better result! Our LLM provides us with a final answer of `1392` which is correct. Finally, as mentioned most LLMs are now trained to use CoT prompting by default. So let's see what happens if we don't explicitly tell the LLM to use CoT.

In [ ]:
system_prompt = """
Be a helpful assistant and answer the user's question, keep it concise to the point and engaging/friendly to user.
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{query}"),
])

pipeline = prompt_template | llm

In [27]:
result = pipeline.invoke({"query": query}).content
display(Markdown(result))

To calculate the total number of keystrokes needed to type the numbers from 1 to 500, we can break it down by the number of digits in the numbers.

1. **Numbers from 1 to 9**: 
   - There are 9 numbers (1 to 9).
   - Each number has 1 digit.
   - Total keystrokes = 9 * 1 = 9.

2. **Numbers from 10 to 99**: 
   - There are 90 numbers (10 to 99).
   - Each number has 2 digits.
   - Total keystrokes = 90 * 2 = 180.

3. **Numbers from 100 to 499**: 
   - There are 400 numbers (100 to 499).
   - Each number has 3 digits.
   - Total keystrokes = 400 * 3 = 1200.

4. **Number 500**: 
   - There is 1 number (500).
   - It has 3 digits.
   - Total keystrokes = 1 * 3 = 3.

Now, we can sum all the keystrokes:

- From 1 to 9: 9 keystrokes
- From 10 to 99: 180 keystrokes
- From 100 to 499: 1200 keystrokes
- From 500: 3 keystrokes

Total keystrokes = 9 + 180 + 1200 + 3 = 1392.

Therefore, the total number of keystrokes needed to type the numbers from 1 to 500 is **1392**.

We almost get the _exact_ same result. The formatting isn't quite as nice but the CoT behavior is clearly there, and the LLM produces the correct final answer!

CoT is useful not only for simple question-answering like this, but is also a fundamental component of many agentic systems which will often use CoT steps paired with tool use to solve very complex problems, this is what we see in OpenAI's current flagship model `o1`. We'll see later in the course how we can do this ourselves.

---